# Data preprocessing

In [1]:
from scripts import preparate_data

Поменяйте пути к данным на нужные, если они отличаются от текущих

In [2]:
question_path = "data/source_data/v2_OpenEnded_mscoco_val2014_questions.json"
answer_path = "data/source_data/v2_mscoco_val2014_annotations.json"

image_source_dir = "data/source_data/all_images"
data_dir = "data/preprocessed_data/data"

In [4]:
preparate_data(
    q_json_path = question_path,
    a_json_path = answer_path,
    img_src_dir = image_source_dir,
    data_dir = data_dir,
    data_size = 5 # Увеличьте размер датасета до нужного
)

Starting data preparation...
Done creating QA dictionary.
Grouping data by image...
Full dataset size: 40504
Dataset size after trimming: 5
Done copying images.
Data preparation completed.


In [3]:
# установим необходимые библиотеки
!pip install -r requirements.txt

# Если используете Google Colab - достаточно лишь этих библиотек
# !pip install accelerate deepspeed peft timm mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 35.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.

# Fine-Tuning openbmb/MINICMP-V-2
Склонируем репозиторий с кодом самой модельки

In [1]:
!git clone https://github.com/OpenBMB/MiniCPM-V.git

Cloning into 'MiniCPM-V'...
remote: Enumerating objects: 1242, done.
remote: Counting objects: 100% (701/701), done.
remote: Compressing objects: 100% (368/368), done.
remote: Total 1242 (delta 520), reused 418 (delta 328), pack-reused 541
Receiving objects: 100% (1242/1242), 191.40 MiB | 15.67 MiB/s, done.
Resolving deltas: 100% (718/718), done.
Updating files: 100% (143/143), done.


Переименуем чтобы не было конфликтов при выполнении команды( на тире в названии ругается)

In [2]:
!mv MiniCPM-V MiniCPM_V

Код ниже нужен лишь в том случае, если используете Google Colab и импортируете данные с вашего Google диска

In [4]:
# подключим Гугл-диск и возьмём оттуда данные для обучения
# from google.colab import drive
# drive.mount('/content/drive')

# !cp /content/drive/MyDrive/data.zip .
# !unzip data.zip -d .

## Код запуска обучения (через командную строку)

### Константы и Гиперпараметры

In [ ]:
import os
import subprocess

from scripts import insert_shebang, insert_code

GPUS_PER_NODE = 1
NNODES = 1
NODE_RANK = 0
MASTER_ADDR = "localhost"
MASTER_PORT = 6001

MODEL = "openbmb/MiniCPM-V-2"
LLM_TYPE = "minicpm"

# Укажите путь к вашим данным
TRAIN_DATA_PATH = f"{data_dir}/train_annot.json"
VALID_DATA_PATH = f"{data_dir}/valid_annot.json"

# Hyperparameters
MAX_STEPS = 110
CHECKPOINT_STEPS = 105
EVAL_STEPS = 5

LR=1e-6
W_DECAY = 0.1
ADAM_BETA2 = 0.95
WARMUP_RATIO = 0.01

OPTIMIZER = "adafactor"

GRAD_ACCUM_STEPS = 8
MAX_LENGTH = 1024
SLICE_NUMS = 3

TRAIN_BATCH_SIZE = 1
VALID_BATCH_SIZE = 1

# target_modules_regex = "\"lm\..*layers\.(?!2|4)\d+\.self_attn\.(q_proj|k_proj)\""
target_modules_regex = "\"llm\..*layers\.\d+\.self_attn\.(q_proj|k_proj)\""

# Аргументы для распределённого запуска
DISTRIBUTED_ARGS = f"""
    --nproc_per_node {GPUS_PER_NODE} \
    --nnodes {NNODES} \
    --node_rank {NODE_RANK} \
    --master_addr {MASTER_ADDR} \
    --master_port {MASTER_PORT}
"""

script_path = "MiniCPM_V/finetune/finetune.py"

output_dir = "output/output_minicpmv2_lora"

Вставим shebang в начала файла (для того, чтобы ОС понимала, какой интерпретатор нужно использовать).

А также отключим распостранение градиента на очень тяжёлый слой в нашей модели(в противном случае, мы не сможем запустить на нашей бесплатной видеокарте T4)

In [ ]:
code_line = "\n    " + "model.base_model.model.llm.model.embed_tokens.weight.requires_grad = False"

insert_shebang(script_path)
insert_code(script_path, code_line, 278)

Теперь запустим обучение, сымитировав вызов через командную строку

In [ ]:
# Делаем файл исполняемым
subprocess.run(['chmod', '+x', script_path])

command = f"""
torchrun {DISTRIBUTED_ARGS} {script_path} \
    --model_name_or_path {MODEL} \
    --llm_type {LLM_TYPE} \
    --data_path {TRAIN_DATA_PATH} \
    --eval_data_path {VALID_DATA_PATH} \
    --optim {OPTIMIZER} \
    --remove_unused_columns false \
    --label_names "labels" \
    --prediction_loss_only false \
    --bf16 true \
    --bf16_full_eval true \
    --fp16 false \
    --fp16_full_eval false \
    --do_train \
    --do_eval \
    --tune_vision false \
    --tune_llm false \
    --use_lora true \
    --lora_target_modules {target_modules_regex} \
    --model_max_length {MAX_LENGTH} \
    --max_slice_nums {SLICE_NUMS} \
    --max_steps {MAX_STEPS} \
    --eval_steps {CHECKPOINT_STEPS} \
    --output_dir {output_dir} \
    --logging_dir {output_dir} \
    --logging_strategy "steps" \
    --per_device_train_batch_size {TRAIN_BATCH_SIZE} \
    --per_device_eval_batch_size {VALID_BATCH_SIZE} \
    --gradient_accumulation_steps {GRAD_ACCUM_STEPS} \
    --evaluation_strategy "steps" \
    --save_strategy "steps" \
    --save_steps {CHECKPOINT_STEPS} \
    --save_total_limit 10 \
    --learning_rate {LR} \
    --weight_decay {W_DECAY} \
    --warmup_ratio {WARMUP_RATIO} \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --gradient_checkpointing true \
    --deepspeed /content/MiniCPM_V/finetune/ds_config_zero2.json \
    --report_to "tensorboard"
"""

result = subprocess.run(command, shell=True, capture_output=True, text=True)

print(result.stdout)
print(result.stderr)

## Train and Valid Loss
Построим графики Loss'a на обучающей и валидационной выборках

In [ ]:
from scripts import get_latest_checkpoint, plot_loss_from_trainer_state

latest_checkpoint = get_latest_checkpoint(output_dir)
plot_loss_from_trainer_state(latest_checkpoint + "/trainer_state.json")

## Saving data(if Google Colab)

In [ ]:
# # Путь к исходной папке, которую нужно сжать
# source_folder = "output/output_minicpmv2_lora"

# # Путь, куда будет перемещён архив
# target_folder = '/content/drive/MyDrive'

# current_datetime = datetime.now().strftime('%Y%m%d_%H%M%S')
# zip_filename = f'{current_datetime}_final_output'
# zip_path = os.path.join(target_folder, zip_filename)

# shutil.make_archive(zip_path, 'zip', source_folder)